# Import Library

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import joblib

In [ ]:
df = pd.read_csv("/content/enhanced_food_data.csv")
df.head()

,item,kalori,protein,lemak,karbohidrat,url,food_name,Cholesterol_g,saturated_fat_g,fiber_g,...,vitamin_b_mcg,vitamin_c_mcg,vitamin_d_mcg,vitamin_e_mcg,weight_management,muscle_development,energy_boost,heart_health,immunity_strength,flavor_profile
0,Ayam Bakar,372,"30,95g","7,14g","17,57g",https://www.fatsecret.co.id/kalori-gizi/rumah-...,Ayam Bakar,0.140,4.5,0.0,...,500.0,0.0,0.10,300.0,7.0,9.0,8.0,6.0,8.0,"asin, asam, manis, pedas, gurih"
1,Ayam Goreng,260,"14,55g","10,76g","21,93g",https://www.fatsecret.co.id/kalori-gizi/umum/a...,Ayam Goreng,0.110,7.5,0.0,...,400.0,0.0,0.10,800.0,3.0,9.0,8.0,3.0,7.0,"goreng, asin, gurih"
2,Ayam Semur,415,"15,98g","18,4g","47,35g",https://www.fatsecret.co.id/kalori-gizi/umum/s...,Ayam Semur,0.045,4.0,0.5,...,5000.0,5.0,0.15,0.8,6.0,9.0,8.0,6.0,7.0,"kuah, asin, manis, gurih"
3,Bakso Ayam,45,"1,57g","1,94g","5,43g",https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bakso Ayam,0.021,0.3,0.1,...,120.0,0.0,0.00,0.0,7.0,9.0,6.0,5.0,8.0,"kuah, asin, gurih"
4,Bubur Ayam,372,"12,39g","36,12g","27,56g",https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bubur Ayam,0.007,0.5,0.5,...,40.0,3.0,0.05,0.2,6.0,4.0,8.0,5.0,6.0,"kuah, goreng, asin, gurih"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item                25 non-null     object 
 1   kalori              25 non-null     int64  
 2   protein             25 non-null     object 
 3   lemak               25 non-null     object 
 4   karbohidrat         25 non-null     object 
 5   url                 25 non-null     object 
 6   food_name           25 non-null     object 
 7   Cholesterol_g       25 non-null     float64
 8   saturated_fat_g     25 non-null     float64
 9   fiber_g             25 non-null     float64
 10  sugars_g            25 non-null     float64
 11  sodium_mg           25 non-null     float64
 12  iron_mg             25 non-null     float64
 13  zinc_mg             25 non-null     float64
 14  calcium_mg          25 non-null     float64
 15  vitamin_b12_mcg     25 non-null     float64
 16  vitamin_a_

# Data preparation

In [ ]:
df_food = df.copy()

In [ ]:
def remove_g_and_convert_to_float(value):
  num_str = value[:-1].replace(',', '.')
  return float(num_str)

columns_to_convert = ['protein', 'lemak', 'karbohidrat']
for col in columns_to_convert:
  df_food[col] = df_food[col].apply(remove_g_and_convert_to_float)

df_food.head()

,item,kalori,protein,lemak,karbohidrat,url,food_name,Cholesterol_g,saturated_fat_g,fiber_g,...,vitamin_b_mcg,vitamin_c_mcg,vitamin_d_mcg,vitamin_e_mcg,weight_management,muscle_development,energy_boost,heart_health,immunity_strength,flavor_profile
0,Ayam Bakar,372,30.95,7.14,17.57,https://www.fatsecret.co.id/kalori-gizi/rumah-...,Ayam Bakar,0.140,4.5,0.0,...,500.0,0.0,0.10,300.0,7.0,9.0,8.0,6.0,8.0,"asin, asam, manis, pedas, gurih"
1,Ayam Goreng,260,14.55,10.76,21.93,https://www.fatsecret.co.id/kalori-gizi/umum/a...,Ayam Goreng,0.110,7.5,0.0,...,400.0,0.0,0.10,800.0,3.0,9.0,8.0,3.0,7.0,"goreng, asin, gurih"
2,Ayam Semur,415,15.98,18.40,47.35,https://www.fatsecret.co.id/kalori-gizi/umum/s...,Ayam Semur,0.045,4.0,0.5,...,5000.0,5.0,0.15,0.8,6.0,9.0,8.0,6.0,7.0,"kuah, asin, manis, gurih"
3,Bakso Ayam,45,1.57,1.94,5.43,https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bakso Ayam,0.021,0.3,0.1,...,120.0,0.0,0.00,0.0,7.0,9.0,6.0,5.0,8.0,"kuah, asin, gurih"
4,Bubur Ayam,372,12.39,36.12,27.56,https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bubur Ayam,0.007,0.5,0.5,...,40.0,3.0,0.05,0.2,6.0,4.0,8.0,5.0,6.0,"kuah, goreng, asin, gurih"


In [ ]:
rename_col = {
    "protein" : "protein_g",
    "lemak" : "lemak_g",
    "karbohidrat" : "karbohidrat_g"
}

df.rename(columns=rename_col)

,item,kalori,protein_g,lemak_g,karbohidrat_g,url,food_name,Cholesterol_g,saturated_fat_g,fiber_g,...,vitamin_b_mcg,vitamin_c_mcg,vitamin_d_mcg,vitamin_e_mcg,weight_management,muscle_development,energy_boost,heart_health,immunity_strength,flavor_profile
0,Ayam Bakar,372,"30,95g","7,14g","17,57g",https://www.fatsecret.co.id/kalori-gizi/rumah-...,Ayam Bakar,0.140,4.50,0.0,...,500.00,0.0,0.10,300.00,7.0,9.0,8.0,6.0,8.0,"asin, asam, manis, pedas, gurih"
1,Ayam Goreng,260,"14,55g","10,76g","21,93g",https://www.fatsecret.co.id/kalori-gizi/umum/a...,Ayam Goreng,0.110,7.50,0.0,...,400.00,0.0,0.10,800.00,3.0,9.0,8.0,3.0,7.0,"goreng, asin, gurih"
2,Ayam Semur,415,"15,98g","18,4g","47,35g",https://www.fatsecret.co.id/kalori-gizi/umum/s...,Ayam Semur,0.045,4.00,0.5,...,5000.00,5.0,0.15,0.80,6.0,9.0,8.0,6.0,7.0,"kuah, asin, manis, gurih"
3,Bakso Ayam,45,"1,57g","1,94g","5,43g",https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bakso Ayam,0.021,0.30,0.1,...,120.00,0.0,0.00,0.00,7.0,9.0,6.0,5.0,8.0,"kuah, asin, gurih"
4,Bubur Ayam,372,"12,39g","36,12g","27,56g",https://www.fatsecret.co.id/kalori-gizi/umum/b...,Bubur Ayam,0.007,0.50,0.5,...,40.00,3.0,0.05,0.20,6.0,4.0,8.0,5.0,6.0,"kuah, goreng, asin, gurih"
5,Cumi Goreng,125,"2,17g","9,9g","15,13g",https://www.fatsecret.co.id/kalori-gizi/umum/c...,Cumi Goreng,0.220,4.50,0.0,...,2500.00,2500.0,0.00,0.10,3.0,8.0,7.0,4.0,7.0,"goreng, asin, gurih"
6,Gado Gado,132,"7,41g","10,9g","7,14g",https://www.fatsecret.co.id/kalori-gizi/umum/g...,Gado Gado,0.045,3.00,3.5,...,150.00,12000.0,0.05,2.00,6.0,7.0,8.0,6.0,8.0,"kuah, goreng, asin, asam, manis, pedas, gurih"
7,Gudeg,127,"4,71g","22,54g","1,82g",https://www.fatsecret.co.id/kalori-gizi/umum/g...,Gudeg,0.030,5.20,3.4,...,25.00,2000.0,0.00,1000.00,4.0,4.0,8.0,3.0,5.0,"kuah, manis, gurih"
8,Gulai Ikan,100,"4,59g","1,9g","12,4g",https://www.fatsecret.co.id/kalori-gizi/umum/g...,Gulai Ikan,0.065,5.50,0.8,...,150.00,800.0,1.50,1.00,6.0,8.0,7.0,6.0,7.0,"kuah, asin, asam, pedas, gurih"
9,Iga Bakar,254,19g,"0,94g","19,1g",https://www.fatsecret.co.id/kalori-gizi/umum/i...,Iga Bakar,0.100,10.00,0.0,...,80.00,0.0,0.00,0.00,3.0,9.0,7.0,3.0,8.0,"asin, asam, manis, pedas, gurih"


In [ ]:
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item                25 non-null     object 
 1   kalori              25 non-null     int64  
 2   protein             25 non-null     float64
 3   lemak               25 non-null     float64
 4   karbohidrat         25 non-null     float64
 5   url                 25 non-null     object 
 6   food_name           25 non-null     object 
 7   Cholesterol_g       25 non-null     float64
 8   saturated_fat_g     25 non-null     float64
 9   fiber_g             25 non-null     float64
 10  sugars_g            25 non-null     float64
 11  sodium_mg           25 non-null     float64
 12  iron_mg             25 non-null     float64
 13  zinc_mg             25 non-null     float64
 14  calcium_mg          25 non-null     float64
 15  vitamin_b12_mcg     25 non-null     float64
 16  vitamin_a_

In [ ]:
numerical_cols_without_goals = df_food.drop(columns = ['weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']).select_dtypes(include=['number']).columns.tolist()
numerical_cols_without_goals

['kalori',
 'protein',
 'lemak',
 'karbohidrat',
 'Cholesterol_g',
 'saturated_fat_g',
 'fiber_g',
 'sugars_g',
 'sodium_mg',
 'iron_mg',
 'zinc_mg',
 'calcium_mg',
 'vitamin_b12_mcg',
 'vitamin_a_mcg',
 'vitamin_b_mcg',
 'vitamin_c_mcg',
 'vitamin_d_mcg',
 'vitamin_e_mcg']

In [ ]:
scaler_nutrients = StandardScaler()
df_food[numerical_cols_without_goals] = scaler_nutrients.fit_transform(df_food[numerical_cols_without_goals])

In [ ]:
# prompt: export scaler_nutrients

joblib.dump(scaler_nutrients, 'scaler_nutrients.joblib')

['scaler_nutrients.joblib']

In [ ]:
scaler_goals = MinMaxScaler()
df_food[['weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']] = scaler_goals.fit_transform(df_food[['weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']])

In [ ]:
df_food.drop(columns=['food_name'], inplace=True)

In [ ]:
df_goals = df_food[['item','weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']]

In [ ]:
df_food_nutrients = df_food.drop(columns=['flavor_profile', 'url', 'weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength'])

In [ ]:
df_food_flavor = df_food[['item', 'flavor_profile']]

In [ ]:
# Inisialisasi TfidfVectorizer
tfidf = TfidfVectorizer()

# Melakukan perhitungan idf pada data cuisine
tfidf.fit(df_food_flavor['flavor_profile'])

# Mapping array dari fitur index integer ke fitur nama
tfidf.get_feature_names_out()

array(['asam', 'asin', 'goreng', 'gurih', 'kuah', 'manis', 'pedas'],
      dtype=object)

In [ ]:
df_food.to_csv('df_food_exported.csv', index=False)

# Recommendation system based on food nutrients

In [ ]:
df_food_nutrients.head()

,item,kalori,protein,lemak,karbohidrat,Cholesterol_g,saturated_fat_g,fiber_g,sugars_g,sodium_mg,iron_mg,zinc_mg,calcium_mg,vitamin_b12_mcg,vitamin_a_mcg,vitamin_b_mcg,vitamin_c_mcg,vitamin_d_mcg,vitamin_e_mcg
0,Ayam Bakar,1.582311,2.756766,-0.485127,0.545641,0.504510,0.110688,-0.822832,0.137797,1.176188,0.417469,0.603571,-0.786995,-0.302239,-0.400908,0.016400,-0.316863,-0.423435,0.092293
1,Ayam Goreng,0.624578,0.633747,-0.221945,0.956039,0.207972,1.071525,-0.822832,-0.739222,0.561027,0.026580,0.603571,-0.786995,-0.499009,-0.505094,-0.079539,-0.316863,-0.423435,1.267883
2,Ayam Semur,1.950012,0.818864,0.333499,3.348771,-0.434527,-0.049451,-0.266864,1.716431,1.176188,-0.494605,-0.095816,-0.786995,-0.646586,-0.400908,4.333654,-0.314761,-0.357847,-0.611180
3,Bakso Ayam,-1.213926,-1.046545,-0.863178,-0.597072,-0.671758,-1.234482,-0.711638,-0.739222,0.407237,-0.885494,-0.532933,-0.786995,-0.695779,-0.817652,-0.348168,-0.316863,-0.554611,-0.613060
4,Bubur Ayam,1.582311,0.354130,1.621781,1.485979,-0.810143,-1.170427,-0.266864,-0.669060,-1.284457,-1.276383,-0.970050,-1.091560,-0.863033,-0.775978,-0.424920,-0.315602,-0.489023,-0.612590


In [ ]:
nutrients_matrix = df_food_nutrients.drop(columns=['item'])

In [ ]:
cosine_sim_nutrients = cosine_similarity(nutrients_matrix)

cosine_sim_nutrients_df = pd.DataFrame(cosine_sim_nutrients, index=df_food_nutrients['item'], columns=df_food_nutrients['item'])

In [ ]:
cosine_sim_nutrients_df

item,Ayam Bakar,Ayam Goreng,Ayam Semur,Bakso Ayam,Bubur Ayam,Cumi Goreng,Gado Gado,Gudeg,Gulai Ikan,Iga Bakar,...,Opor Ayam,Papeda,Rawon,Rendang,Sate Ayam,Soto Ayam,Sop Buntut,Telur Dadar,Telur Rebus,Ikan Bakar
item,,,,,,,,,,,,,,,,,,,,,
Ayam Bakar,1.000000,0.611045,0.390238,-0.279374,0.132793,-0.245333,-0.326499,-0.281884,-0.320225,0.515298,...,-0.109046,-0.471267,-0.043670,0.186108,-0.110777,0.380690,-0.384499,-0.165561,-0.273425,0.265490
Ayam Goreng,0.611045,1.000000,0.251935,-0.140403,0.124761,-0.058065,-0.514290,-0.303564,-0.282377,0.417030,...,0.238976,-0.289345,-0.034034,0.260922,0.047557,0.203256,-0.327358,-0.014645,-0.107633,-0.086187
Ayam Semur,0.390238,0.251935,1.000000,-0.225135,0.295497,0.232957,-0.127223,-0.050078,-0.253820,0.109721,...,-0.118275,-0.312057,-0.114133,-0.014306,-0.142213,0.354116,-0.424885,-0.312436,-0.358166,-0.153444
Bakso Ayam,-0.279374,-0.140403,-0.225135,1.000000,0.161275,0.136845,-0.199962,-0.213306,0.098827,-0.340885,...,0.127960,0.689450,0.884779,-0.471358,0.098049,0.240780,0.331056,-0.328477,-0.194797,-0.181802
Bubur Ayam,0.132793,0.124761,0.295497,0.161275,1.000000,-0.060350,-0.231618,-0.226807,-0.464450,-0.357735,...,0.166227,0.461359,0.174034,-0.493090,-0.616635,0.883204,-0.438127,-0.494682,-0.362601,-0.241240
Cumi Goreng,-0.245333,-0.058065,0.232957,0.136845,-0.060350,1.000000,-0.062753,-0.388697,0.134156,-0.015184,...,0.122847,0.180084,-0.019419,0.064609,-0.164801,-0.198624,-0.037577,0.075545,0.181369,-0.154864
Gado Gado,-0.326499,-0.514290,-0.127223,-0.199962,-0.231618,-0.062753,1.000000,0.534776,0.029569,-0.279302,...,-0.198831,-0.162800,-0.201643,-0.071767,0.047877,-0.208618,0.111199,0.000378,-0.017951,-0.175390
Gudeg,-0.281884,-0.303564,-0.050078,-0.213306,-0.226807,-0.388697,0.534776,1.000000,-0.181274,-0.201174,...,-0.249534,-0.119194,-0.213282,-0.113617,0.475424,-0.300224,-0.098101,-0.076734,-0.139413,-0.129407
Gulai Ikan,-0.320225,-0.282377,-0.253820,0.098827,-0.464450,0.134156,0.029569,-0.181274,1.000000,0.152673,...,0.075452,-0.195680,0.202925,0.304201,-0.047873,-0.366513,0.248006,0.262793,0.200248,0.442402


In [ ]:
def getDetailNutrients(item):
  return df_food_nutrients[df_food_nutrients['item'] == item]

In [ ]:
def recipe_recommendations_based_nutrients(foodName, similarity_data=cosine_sim_nutrients_df, k=5):
    sim_scores = similarity_data[foodName]

    recommendations = (
        sim_scores
        .sort_values(ascending=False)
        .drop(foodName, errors='ignore')
        .head(k)
        .reset_index()
        .rename(columns={foodName: 'similarity_score'})
    )
    return recommendations

In [ ]:
getDetailNutrients('Telur Dadar')

,item,kalori,protein,lemak,karbohidrat,Cholesterol_g,saturated_fat_g,fiber_g,sugars_g,sodium_mg,iron_mg,zinc_mg,calcium_mg,vitamin_b12_mcg,vitamin_a_mcg,vitamin_b_mcg,vitamin_c_mcg,vitamin_d_mcg,vitamin_e_mcg
22,Telur Dadar,-0.803469,-0.300899,-0.973685,-0.498238,2.975663,0.203569,-0.822832,-0.570834,-0.746191,0.599884,-0.017135,1.832262,0.455326,2.620488,-0.012382,-0.316863,2.855959,2.584543


In [ ]:
recipe_recommendations_based_nutrients('Telur Dadar')

,item,similarity_score
0,Telur Rebus,0.963257
1,Martabak Telur,0.410065
2,Ikan Bakar,0.354317
3,Gulai Ikan,0.262793
4,Sop Buntut,0.203372


In [ ]:
# Save the model artifacts
# joblib.dump(scaler_nutrients, 'scaler_nutrients.joblib')
# joblib.dump(scaler_goals, 'scaler_goals.joblib')
# joblib.dump(tfidf, 'tfidf_vectorizer.joblib')
joblib.dump(cosine_sim_nutrients_df, 'cosine_sim_nutrients_df.joblib')

# To use this on a user device, you would need to load these files and implement the recommendation logic.
# You would also need to make the df_food_nutrients and df_food_flavor data available for loading on the device.
# This usually involves saving these dataframes to a format like CSV or Parquet and loading them on the device.
# For instance:
# df_food_nutrients.to_csv('df_food_nutrients.csv', index=False)
# df_food_flavor.to_csv('df_food_flavor.csv', index=False)

# Then on the user device:
# import joblib
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Load the saved artifacts
# scaler_nutrients = joblib.load('scaler_nutrients.joblib')
# scaler_goals = joblib.load('scaler_goals.joblib')
# tfidf = joblib.load('tfidf_vectorizer.joblib')
# cosine_sim_nutrients_df = joblib.load('cosine_sim_nutrients_df.joblib')
# df_food_nutrients = pd.read_csv('df_food_nutrients.csv')
# df_food_flavor = pd.read_csv('df_food_flavor.csv')

# # You would then define or load the necessary functions like getDetailNutrients and recipe_recommendations_based_nutrients
# # and use them with the loaded data and objects.

# # Example of a simplified recommendation function on the user device
# def recipe_recommendations_based_nutrients_inference(foodName, similarity_data, k=5):
#     if foodName not in similarity_data.index:
#         return pd.DataFrame(columns=['item', 'similarity_score']) # Or handle as appropriate
#     sim_scores = similarity_data[foodName]
#     recommendations = (
#         sim_scores
#         .sort_values(ascending=False)
#         .drop(foodName, errors='ignore')
#         .head(k)
#         .reset_index()
#         .rename(columns={foodName: 'similarity_score'})
#     )
#     return recommendations

# # Example usage on the user device
# # recommendations = recipe_recommendations_based_nutrients_inference('Telur Dadar', cosine_sim_nutrients_df)
# # print(recommendations)



['cosine_sim_nutrients_df.joblib']

# Recommendation system based on Flavor

In [ ]:
keyword_matrix = tfidf.fit_transform(df_food_flavor['flavor_profile'].fillna(''))

In [ ]:
cosine_sim_keyword = cosine_similarity(keyword_matrix)

cosine_sim_keyword_df = pd.DataFrame(cosine_sim_keyword, index=df_food_flavor['item'], columns=df_food_flavor['item'])

In [ ]:
def getDetailFlavor(item):
  return df_food_flavor[df_food_flavor['item'] == item]

In [ ]:
def recipe_recommendations_based_keyword(recipeId, similarity_data=cosine_sim_keyword_df, items=df_food_flavor[['item', 'flavor_profile']], k=5):
    sim_scores = similarity_data[recipeId]

    recommendations = (
        sim_scores
        .sort_values(ascending=False)
        .drop(recipeId, errors='ignore')
        .head(k)
        .reset_index()
        .rename(columns={recipeId: 'similarity_score'})
        .merge(items, on='item')
    )
    return recommendations

In [ ]:
getDetailFlavor('Telur Dadar')

,item,flavor_profile
22,Telur Dadar,"goreng, asin, gurih"


In [ ]:
recipe_recommendations_based_keyword('Telur Dadar')

,item,similarity_score,flavor_profile
0,Ayam Goreng,1.000000,"goreng, asin, gurih"
1,Cumi Goreng,1.000000,"goreng, asin, gurih"
2,Martabak Telur,1.000000,"goreng, asin, gurih"
3,Bubur Ayam,0.811969,"kuah, goreng, asin, gurih"
4,Mie Goreng,0.811969,"goreng, asin, manis, gurih"


In [ ]:
joblib.dump(cosine_sim_keyword_df, 'cosine_sim_nutrients_df.joblib')

['cosine_sim_nutrients_df.joblib']

# Recommendation system based on goals

In [ ]:
goals_cols = ['weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']

In [ ]:
df_goals.head()

,item,weight_management,muscle_development,energy_boost,heart_health,immunity_strength
0,Ayam Bakar,0.833333,1.000,0.666667,0.571429,1.000000
1,Ayam Goreng,0.166667,1.000,0.666667,0.142857,0.833333
2,Ayam Semur,0.666667,1.000,0.666667,0.571429,0.833333
3,Bakso Ayam,0.833333,1.000,0.000000,0.428571,1.000000
4,Bubur Ayam,0.666667,0.375,0.666667,0.428571,0.666667


In [ ]:
def recipe_recommendations_based_goals(userGoals, items=df_goals[['item', 'weight_management', 'muscle_development', 'energy_boost', 'heart_health', 'immunity_strength']], k=5):
    # scale the goals
    userGoals = scaler_goals.transform([userGoals])

    # create user goals
    df_user_goals = pd.DataFrame(userGoals, columns=goals_cols)
    df_user_goals['item'] = 'userGoals'

    df_temp = df_goals.copy()
    df_temp = pd.concat([df_temp, df_user_goals], ignore_index=True)

    goals_matrix = df_temp.drop(columns=['item'])

    cosine_sim_goals = cosine_similarity(goals_matrix)
    cosine_sim_goals_df = pd.DataFrame(cosine_sim_goals, index=df_temp['item'], columns=df_temp['item'])

    sim_scores = cosine_sim_goals_df['userGoals']

    recommendations = (
        sim_scores
        .sort_values(ascending=False)
        .drop(userGoals, errors='ignore')
        .head(k)
        .reset_index()
        .rename(columns={'userGoals': 'similarity_score'})
        .merge(items, on='item')
    )

    recommendations[goals_cols] = scaler_goals.inverse_transform(recommendations[goals_cols])
    return recommendations

In [ ]:
joblib.dump(scaler_goals, 'scaler_goals.joblib')
joblib.dump(df_goals, 'df_goals.joblib')

['df_goals.joblib']

In [ ]:
"""
  userGoals index :
  0 = weight management
  1 = muscle development
  2 = energy boost
  3 = heart health
  4 = immunity strength

  the scale is from 0 - 10
"""
userGoals = [10, 1, 3, 7, 1]
recipe_recommendations_based_goals(userGoals)

,item,similarity_score,weight_management,muscle_development,energy_boost,heart_health,immunity_strength
0,Soto Ayam,0.502273,7.0,7.0,6.0,6.0,7.0
1,Ikan Bakar,0.419173,8.0,9.0,7.0,9.0,8.0
2,Bakso Ayam,0.404819,7.0,9.0,6.0,5.0,8.0
3,Telur Rebus,0.387726,8.0,9.0,7.0,7.0,8.0
